# Hello, Chameleon

In this notebook, you will pick up where you left off after creating a Chameleon account, joining a Chameleon project, and preparing key pair. Now, you will learn how to:

-   Reserve resources in Chameleon
-   Access your reserved resources over SSH
-   Execute commands on your resources
-   Retrieving files saved on Chameleon resources
-   Extend your Chameleon lease (in case you need more time) or delete it (in case you finish early)

## Exercise: reserve resources

Whenever you run an experiment on Chameleon, you will

1.  Open a Python notebook, which includes commands to reserve and configure the resources (VMs, bare metal servers, or networks) that you need for your experiment. Run these commands.
2.  Wait until the resources in your experiment are ready to log in.
3.  Log in to the resources and run your experiment (either by executing commands in the notebook, or by using SSH in a terminal and running commands in those SSH sessions).

Also, when you finish an experiment and have saved all the data somewhere safe, you will *delete* the resources in your experiment to free them for use by other experimenters.

In this exercise, we will reserve a single virtual machine on Chameleon, and practice logging in to execute commands on this VM.

First, we will need to initialize the environment - tell it what Chameleon project to associate our experiment with.

You should already be a part of a Chameleon project, which has a project ID in the form “CHI-XXXXX”. If you don’t know your project ID, you can find it by logging in to the Chameleon web portal, and checking your [dashboard](https://chameleoncloud.org/user/dashboard/). When you run the next cell, you will see a drop-down menu for selecting your project.

We will also indicate which Chameleon site we want to use. Since this experiment uses a virtual machine, the site will be KVM@TACC - the only Chameleon site that supports VMs.

In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import hardware
from chi import network

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")
username = os.getenv('USER') # all exp resources will have this suffix

Next, we’ll give our resource a name. Every resource in a project should have a unique name, so we will include a username, as well as a description of the experiment, in the name.

In [ ]:
exp_name = "hello_chameleon"
server_name = f"{exp_name}-{username}"
lease_name = f"{exp_name}-{username}"

Now we are ready to ask Chameleon to allocate a resource to us! For a VM, we specify the “flavor” or size of the resource (in terms of CPU, memory, and storage) and the operating system image that we want to have pre-installed.

First we will reserve the VM instance for 6 hours, starting now:

In [ ]:
l = lease.Lease(lease_name, duration=datetime.timedelta(hours=6))
l.add_flavor_reservation(id=chi.server.get_flavor_id("m1.small"), amount=1)
l.submit(idempotent=True)

In [ ]:
l.show()

then we can launch it:

In [ ]:
flavor_name = "m1.small"
image_name = "CC-Ubuntu24.04"
s = server.Server(
    name=server_name,
    image_name=image_name,
    flavor_name=l.get_reserved_flavors()[0].name
)
s.submit(idempotent=True)

Once the resource is allocated and ready, we will associate a network address to it, so that we can log in to the resource over the Internet using the SSH protocol.

In [ ]:
s.associate_floating_ip()

In [ ]:
reserved_fip = s.get_floating_ip()
print(reserved_fip)

There’s one more step before we can log in to the resource - by default, all connections to VM resources are blocked, as a security measure. We will need to add a “security group” that permits SSH connections to our project (if it does not already exist), then attach this security group to our VM resource.

In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")
if sg_list: # allow-ssh already exists
    sg = sg_list[0]
else:       # create allow-ssh
    sg = network.SecurityGroup({"name": "allow-ssh", "description": "Enable SSH traffic on TCP port 22"})
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()
s.add_security_group(sg.id)

That’s all we need to do to prepare a resource to log in! Run the following cell - when it returns, it means that the VM resource is ready for you to log in.

In [ ]:
s.check_connectivity()

## Exercise: log in to resources and execute commands

In this exercise, we’ll practice running commands on the VM resource in three ways:

-   by opening an SSH session in the terminal inside this Jupyter environment, and running commands in that session,
-   by opening an SSH session in a local terminal and running commands in that session.
-   by using the `python-chi` Python interface to execute commands from within this Python notebook.

### Log in over SSH from Jupyter environment

One of the easiest ways to log in to your VM is to open a shell inside the Jupyter environment, and log in over SSH from that shell.

In the Chameleon JupyterHub environment, click File \> New \> Terminal. This will open another tab in the Jupyter environment, with a shell session.

Now, run this cell to get the SSH login command. Copy the output of the cell:

In [ ]:
print(f"ssh cc@{reserved_fip}")

then switch to your terminal shell tab, paste the SSH login command, and hit Enter.

The first time you log in to each new host, you may see a warning similar to the following:

``` shell
The authenticity of host "129.114.26.xx (129.114.26.xx)" cannot be established.
ED25519 key fingerprint is SHA256:1fcbGrgLDdOeorauhz3CTyhmFqOHsrEWlu0TZ6yGoDM.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])?
```

and you will have to type the word *yes* and hit Enter to continue.

Then, you’ll be logged in! To validate that you are logged in to the remote host, and not running commands directly in the Jupyter shell environment, run

``` shell
hostname
```

and verify that the output starts with “hello-chameleon”. (This is the hostname we assigned to our VM resource!)

### Log in over SSH from local terminal

To log in to the VM over SSH from your local terminal, you will follow a similar process:

-   open the terminal application *installed on your computer*,
-   run the cell below, which will print an SSH login command,
-   copy this command and make any necessary modifications (if needed, as described in the following cell),
-   paste it into your terminal and hit Enter.

In this case, you will specify the key location as part of the SSH command. These instructions assume that, as described in the previous steps, you have created a key pair named `id_rsa_chameleon`, put it in the default `.ssh` subdirectory in your home directory, and uploaded it to the KVM@TACC web interface.

In [ ]:
print(f"ssh -i ~/.ssh/id_rsa_chameleon cc@{reserved_fip}")

If your Chameleon key is in a different location, or has a different name, then you may need to modify the `~/.ssh/id_rsa_chameleon` part of this command to point to *your* key.

The first time you log in to each new host, your computer may display a warning similar to the following:

``` shell
The authenticity of host "129.114.26.xx (129.114.26.xx)" cannot be established.
ED25519 key fingerprint is SHA256:1fcbGrgLDdOeorauhz3CTyhmFqOHsrEWlu0TZ6yGoDM.
This key is not known by any other names
Are you sure you want to continue connecting (yes/no/[fingerprint])?
```

and you will have to type the word *yes* and hit Enter to continue.

If you have specified your key path and other details correctly, it won’t ask you for a password when you log in to the resource. (It may ask for the passphrase for your private key if you’ve set one.)

Let’s practice running a command in this remote session. Copy and paste the following command into the SSH terminal, to create a file and populate it with a “hello” message:

``` shell
echo "Hello from $(hostname)" > hello.txt
```

then check the file contents:

``` shell
cat hello.txt
```

Now we will use this file “hello.txt” in a later exercise, when we want to practice transferring files between the remote host and our own laptop!

### Using `python-chi` to execute commands on the remote host

Finally, it’s useful to know that we can also execute commands over SSH on the remote instance, directly from a Python notebook! The following cell shows an example, where we run the `hostname` command using the `python-chi` library:

In [ ]:
s.execute(f"hostname") 

## Exercise: transfer files to and from resources

While working on a remote host, we will often want to transfer files from the remote host to our local filesystem, or vice versa.

To move data back and forth between a terminal and a remote system that you access with *ssh*, we can use *scp*. The syntax is:

``` shell
scp [OPTIONS] SOURCE DESTINATION
```

where `SOURCE` is the full address of the location where the file is currently llocated, and `DESTINATION` is the address of the location that you want to copy a file to.

### Transfering files through the local terminal

We previously generated a file on the remote VM, “hello.txt”. Now, we’ll use `scp` to transfer the file from the remote host to our laptop, make a change to it, then transfer it back.

You will run the `scp` command from your *local* terminal, not on the remote host. If you are still logged in over SSH to the remote host, type

``` shell
exit
```

to return to your local terminal. Check the terminal *prompt* and make sure it reflects that you are executing commands at your local terminal, and not on the Chameleon VM.

Then, we’ll need to generate an `scp` command to run, including:

-   the location of the key you use to SSH into the remote host, e.g. `~/.ssh/id_rsa_chameleon`
-   the username you use to SSH into the remote host, `cc` in this case
-   the IP address or hostname you use to SSH into the remote host
-   the location of the file you want to copy on the remote host, which is `/home/cc/hello.txt`
-   and the location on your laptop to which you want to copy the file. We will copy it to the same location from which you run the scp command (`.` is shorthand for “my current working directory”),

Run the cell below, to generate the `scp` command:

In [ ]:
print(f'scp -i ~/.ssh/id_rsa_chameleon cc@{reserved_fip}:/home/cc/hello.txt .')


Copy the command that is printed by the cell above, and make any changes if necessary (e.g. to the key location or name, or to the location in your local filesystem to which the file should be transferred). Then, execute it in your *local* shell. (Note that the `.` at the end is part of the command - don’t omit this part!)

The output of this command should show that the file is transferred to your local filesystem:

``` text
hello.txt                       100%    1KB     0.1KB/s   00:00
```

When you have successfully transferred “hello.txt” from the remote host to your laptop, locate it in your local filesystem and open it in your preferred text editor. Make a change (any change!) to the file and save it.

Then, we’ll transfer it back to the remote host! To transfer it back to the remote host, the `SOURCE` argument will become the location of the file in the local filesystem, and the `DESTINATION` will become the location to which the file should be transferred on the remote VM.

Use the cell below to generate the `scp` command to transfer the file *to* the remote host:

In [ ]:
print(f'scp -i ~/.ssh/id_rsa_chameleon hello.txt cc@{reserved_fip}:/home/cc/')


Copy the command that is printed by the cell above, and make any changes (e.g. to the key location or name, or to the location in your local filesystem from which the file should be transferred). Then, execute the command in your *local* shell.

The output of this command should show that the file is transferred to the remote filesystem:

``` text
hello.txt                       100%    1KB     0.1KB/s   00:00
```

To validate that the changes you made locally are now reflected in the version of the file that is on the remote host, use the SSH command from the previous section to log in to the remote host again, and run

``` shell
cat hello.txt
```

in the SSH session. Verify that your changes appear in the output.

## Exercise: Hello, Linux

If you are not very comfortable working at the Linux terminal, then before you delete your virtual machine, you can take the opportunity to learn a few basic commands, following the instructions in: [Hello, Linux](https://teaching-on-testbeds.github.io/hello-linux/index-chi)

## Exercise: delete resources

Chameleon is a shared facility, and it is important to be mindful of your resource usage and to “free” resources for use by other experimenters when you are finished with them. Your resource will be deleted automatically at the end of your lease, but if you finish sooner, you should delete the compute instance and the lease.

In the cell below, uncomment both lines of code, then run the cell to free

-   the VM and the network address you attached to it.
-   and the reservation.

Note that removing the resources will revoke your access to them, and all the information stored on them will be erased. Therefore, ensure that you have saved all your work before deleting the resources.

In [ ]:
# s.delete()
# l.delete()

Alternatively, you can delete your instance using the GUI:

-   From the [Chameleon website](https://chameleoncloud.org/), click on “Experiment \> KVM@TACC” in the menu (since that is the site that our instance is on).
-   Select “Instances” from the menu on the left side.
-   Find your instance in the list. If the project that you are part of has many instances, you can filter by name to make it easier to find yours: change the filter criteria to “Instance Name”, put part of your instance name in the text input field, and click “Filter”.
-   Check the box next to *your* instance (make sure not to select someone else’s!)
-   and press the red “Delete Instances” button.

and you can similarly delete a lease using the GUI:

-   Select “Leases” from the menu on the left side.
-   Find your lease in the list.
-   Check the box next to *your* lease (make sure not to select someone else’s!)
-   and press the red “Delete Lease” button.